# News Bot Dissemination

## Collection

In [1]:
import tweepy
import pandas as pd
import numpy as np
from time import sleep

CONSUMER_KEY = "----"
CONSUMER_SECRET = "---"
ACCESS_TOKEN = "-----"
ACCESS_TOKEN_SECRET = "---"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)
api.me()

from twarc import Twarc
t= Twarc(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Fake news dictionary
site_dir = {'abcnews.com.co':'ABCnews.co',
            "AmericanNews.com":"American News",
            "BigAmericanNews.com" : "Big American News",
            "ChristWire.org" : "ChristWire",
            "CivicTribune.com" : "CivicTribune",
            "CreamBMP.com": "CreamBMP",
            "thedcgazette.com":"the DC Gazette",
            "dcclothesline.com": "DCclothesline",
            "DerfMagazine.com": "Derf Mag",
            "DrudgeReport.com.co": "DrudgeReport.co",
            "DuhProgressive.com": "DuhProgressive",
            "empirenews.net":"Empire News",
            "EmpireNews.com":"Empire News",
            "EnduringVision.com": "Enduring Vision",
            "mediamass.net": "MediaMass",
            "NationalReport.net": "NationalReport",
            "NewsBiscuit.com":"NewsBiscuit",
            "News-Hound.com": "NewsHound",
            "NewsMutiny.com": "News Mutiny",
            "PoliticalEars.com" : "PoliticalEars",
            "Private-eye.co.uk": "Private-Eye",
            "RealNewsRightNow.com": "RealNews",
            "Sprotspickle.com": "Sprotspickle",
            "TheNewsNerd.com":"TheNewsNerd",
            "TheUsPatriot.com": "USPatriot",
            "WitScience.org": "WitScience",
            "70news.wordpress.com":"70 News",
            "worldnewsdailyreport.com": "World News Daily",
           "yournewswire.com": "tYourNewsWire",
           "rilenews.com": "RileNews",
           }

# Collection

temp = []

for website in site_dir.keys():
    for tweet in t.search(website):
        tweet['website'] = website
        temp.append(tweet)

        
df = pd.DataFrame(temp)
df.to_csv('full_tweets.csv')


KeyboardInterrupt: 

### Processing

In [6]:

df_to_tweet = pd.DataFrame(df[['id_str','website','user','retweeted_status']])

def process_retweeted_status(x):
    if x != x:
        return False
    else:
        return float('nan')
df_to_tweet['retweet'] = df_to_tweet['retweeted_status'].apply(process_retweeted_status)
df_to_tweet['screen_name'] = df_to_tweet['user'].apply(lambda x: '@'+x['screen_name'])
df_to_tweet['website_name'] = df_to_tweet['website'].apply(lambda x: site_dir[x])
del df_to_tweet['user']
del df_to_tweet['website']
del df_to_tweet['retweeted_status']

df_to_tweet = df_to_tweet.sort_values(['screen_name','website_name'])
df_to_tweet['cum_count'] = df_to_tweet.groupby(['screen_name','website_name']).cumcount()

def make_nan(x):
    if float(x) == 0:
        return 0
    else:
        return float('nan')

df_to_tweet['cum_count'] = df_to_tweet['cum_count'].apply(make_nan)
df_to_tweet = df_to_tweet.dropna()
df_to_tweet = df_to_tweet.sort_index()

sample_size = 720
random_index = sorted(set(list(np.random.randint(0,len(df_to_tweet),sample_size))))
df_to_tweet_final = pd.DataFrame(df_to_tweet.iloc[random_index])
df_to_tweet_final['sent'] = False
df_to_tweet_final = df_to_tweet_final.reset_index()
df_to_tweet_final = df_to_tweet_final.drop_duplicates('screen_name')
df_to_tweet_final.to_csv('to_tweet_sample1.csv')


# Sending out tweets

Using the fields in 'df_to_tweet', tweet out a message to each user.

In [30]:
df['Macron'] = False
for index,row in df.iterrows():
    hashtags_list = []
    for h in row['entities']['hashtags']:
        if h['text'].find('Macron') > -1:
            df.loc[index,'Macron'] = True

In [36]:
df['bio'] = 0
for index,row in df.iterrows():
    df.loc[index,'bio'] = row['user']['description']

In [45]:
count = 0
df_drop = df.drop_duplicates('bio')
for index,row in df_drop.iterrows():
    if row['bio'].find('Trump') > -1:
        count += 1
print(count)

1403


In [46]:
count/len(df_drop.index)

0.09006869101880978

In [52]:
df['user']['statuses_count']

{'contributors_enabled': False,
 'created_at': 'Tue Dec 16 05:44:25 +0000 2014',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'Just Your Basic Nasty Woman',
 'entities': {'description': {'urls': []}},
 'favourites_count': 1136,
 'follow_request_sent': False,
 'followers_count': 29,
 'following': False,
 'friends_count': 189,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 2924374097,
 'id_str': '2924374097',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'en',
 'listed_count': 0,
 'location': 'Portland, OR',
 'name': 'Ruby Twill',
 'notifications': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/2924374097/1493404779',
 'profile_image_url': 'http://pbs.t

In [12]:
tweet_df = []
for index,row in df_to_tweet_final.iterrows():
    if row['sent'] == True:
        pass
    else:
        random_wait = np.random.randint(90,150,1)[0]
        message = 'Hi {}, how reliable do you feel {} is on a scale of 1-5? (1 unreliable, 5 reliable).'.format(row['screen_name'],row['website_name'])
        tweet = api.update_status(message,in_reply_to_status_id=row['id_str'])
        tweet_df.append(tweet)
        df_to_tweet_final.iloc[[index],[6]] = True
        sleep(random_wait)

tweet_df = pd.DataFrame(tweet_df)
tweet_df.to_csv('sent_tweets.csv')

TweepError: [{'code': 261, 'message': 'Application cannot perform write actions. Contact Twitter Platform Operations through https://support.twitter.com/forms/platform'}]

# Code for converting CSV

In [4]:
site_dir = {'abcnews.com.co':'ABCnews.co',
            "AmericanNews.com":"American News",
            "BigAmericanNews.com" : "Big American News",
            "ChristWire.org" : "ChristWire",
            "CivicTribune.com" : "CivicTribune",
            "CreamBMP.com": "CreamBMP",
            "thedcgazette.com":"the DC Gazette",
            "dcclothesline.com": "DCclothesline",
            "DerfMagazine.com": "Derf Mag",
            "DrudgeReport.com.co": "DrudgeReport.co",
            "DuhProgressive.com": "DuhProgressive",
            "empirenews.net":"Empire News",
            "EmpireNews.com":"Empire News",
            "EnduringVision.com": "Enduring Vision",
            "mediamass.net": "MediaMass",
            "NationalReport.net": "NationalReport",
            "NewsBiscuit.com":"NewsBiscuit",
            "News-Hound.com": "NewsHound",
            "NewsMutiny.com": "News Mutiny",
            "PoliticalEars.com" : "PoliticalEars",
            "Private-eye.co.uk": "Private-Eye",
            "RealNewsRightNow.com": "RealNews",
            "Sprotspickle.com": "Sprotspickle",
            "TheNewsNerd.com":"TheNewsNerd",
            "TheUsPatriot.com": "USPatriot",
            "WitScience.org": "WitScience",
            "70news.wordpress.com":"70 News",
            "worldnewsdailyreport.com": "World News Daily",
           "yournewswire.com": "YourNewsWire",
           "rilenews.com": "RileNews",
           }
for key in list(site_dir.keys()):
    print(key)

DuhProgressive.com
NewsMutiny.com
70news.wordpress.com
ChristWire.org
worldnewsdailyreport.com
RealNewsRightNow.com
CreamBMP.com
DerfMagazine.com
TheUsPatriot.com
EnduringVision.com
DrudgeReport.com.co
AmericanNews.com
PoliticalEars.com
WitScience.org
BigAmericanNews.com
rilenews.com
NewsBiscuit.com
mediamass.net
News-Hound.com
yournewswire.com
EmpireNews.com
abcnews.com.co
thedcgazette.com
NationalReport.net
Sprotspickle.com
empirenews.net
dcclothesline.com
CivicTribune.com
TheNewsNerd.com
Private-eye.co.uk


In [4]:
import tweepy
import pandas as pd
import numpy as np
from time import sleep

CONSUMER_KEY = "lBSmQ8m3KRaCCnVRO42MqO4vQ"
CONSUMER_SECRET = "06ci6sqCvse2T3mNM3f5Aat2srWqohxHXqRf0fBdMhXylqMCGT"
ACCESS_TOKEN = "851475089971458048-XkFWKdxlI1MIxz7vQjZnZUm9fia6Jxs"
ACCESS_TOKEN_SECRET = "d4U5WtcKpmTIoWgwshMHyVScofXSJDH3eHRAxBfwWNJSY"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)
api.me()

search = []
for ind in range(20):
    results = api.home_timeline(page=ind)
    search.append(results)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]